In [43]:
import os
import re
import pandas as pd
import numpy as np

In [44]:
total_tfidf_df_list = list()

for i in range(0,1095):
    t_index_list = []
    tfidf_list = []
    
    path = "./tf_idf_vector/" + str(i + 1) + ".txt"
    with open(path, 'r', encoding='utf-8') as file:
        # 跳過Header
        next(file)
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                t_index, tfidf = int(parts[0]), float(parts[1])
                t_index_list.append(t_index)
                tfidf_list.append(tfidf)

    single_doc_data = {'t_index': t_index_list, 'tf_idf': tfidf_list}
    single_doc = pd.DataFrame(single_doc_data)
    total_tfidf_df_list.append(single_doc)

In [45]:
total_tfidf_df_list[:500]

[     t_index    tf_idf
 0      11970  0.005025
 1       4970  0.003534
 2        338  0.002228
 3       5710  0.005692
 4       1955  0.003865
 ..       ...       ...
 120     7207  0.002693
 121     6102  0.005632
 122    10426  0.007297
 123     7561  0.008118
 124     7217  0.003042
 
 [125 rows x 2 columns],
      t_index    tf_idf
 0      11220  0.004213
 1       7217  0.015985
 2       7644  0.008873
 3      10798  0.009644
 4       8106  0.003673
 ..       ...       ...
 122     8648  0.006897
 123     9207  0.008293
 124      217  0.007109
 125     6521  0.006232
 126     2762  0.004239
 
 [127 rows x 2 columns],
      t_index    tf_idf
 0       8262  0.001675
 1      10377  0.006089
 2       6038  0.002125
 3       1713  0.003906
 4       8136  0.003735
 ..       ...       ...
 334     8449  0.002125
 335    11930  0.001468
 336     4749  0.002289
 337    10772  0.001762
 338     3762  0.002029
 
 [339 rows x 2 columns],
      t_index    tf_idf
 0      12208  0.019903
 1     

In [46]:
total_tfidf_df_list[0]

t_index    tf_idf
0      11970  0.005025
1       4970  0.003534
2        338  0.002228
3       5710  0.005692
4       1955  0.003865
..       ...       ...
120     7207  0.002693
121     6102  0.005632
122    10426  0.007297
123     7561  0.008118
124     7217  0.003042

[125 rows x 2 columns]

In [47]:
total_tfidf_df_list[0]['tf_idf']

0      0.005025
1      0.003534
2      0.002228
3      0.005692
4      0.003865
         ...   
120    0.002693
121    0.005632
122    0.007297
123    0.008118
124    0.003042
Name: tf_idf, Length: 125, dtype: float64

In [6]:
# 計算其unit vector
def norm(tfidf_List):
    norm_vector = sum(tfidf ** 2 for tfidf in tfidf_List) ** 0.5
    return norm_vector

In [7]:
def cosine(doc1_df, doc2_df):
    # 以t_index做outer merge
    merged_df = pd.merge(doc1_df, doc2_df, on='t_index', how='outer').fillna(0)
    
    # 轉換為unit vector
    doc1_norm = norm(merged_df['tf_idf_x'].to_list())
    doc2_norm = norm(merged_df['tf_idf_y'].to_list())
    
    # 计算cosine similarity
    similarity = np.dot(merged_df['tf_idf_x']/doc1_norm, merged_df['tf_idf_y']/doc2_norm)
    
    return similarity

In [8]:
column_c = list(range(0,1095))
row_c = list(range(0,1095))

In [18]:
AS = cosine(total_tfidf_df_list[0],total_tfidf_df_list[17])
AS

0.3341522060559187

In [11]:
C = pd.DataFrame(columns=column_c, index=row_c)
for i in range(0,1095):
    for j in range(i,1095):
        C.iat[i,j] = cosine(total_tfidf_df_list[i],total_tfidf_df_list[j])
        C.iat[j,i] = C.iat[i,j]

In [12]:
C

0         1         2         3         4         5         6     \
0          1.0  0.202227  0.302513  0.218558  0.262001  0.245064  0.293049   
1     0.202227       1.0  0.210836  0.233014   0.25783  0.185268   0.20792   
2     0.302513  0.210836       1.0  0.337723  0.312437  0.401215  0.397163   
3     0.218558  0.233014  0.337723       1.0  0.327056  0.210253   0.32045   
4     0.262001   0.25783  0.312437  0.327056       1.0  0.321078  0.298253   
...        ...       ...       ...       ...       ...       ...       ...   
1090  0.021772  0.000689  0.017432   0.00202  0.040304  0.016327  0.025611   
1091  0.025764  0.011381  0.033008  0.013693  0.018786  0.067798  0.031362   
1092  0.034973  0.027174  0.044105  0.020582  0.021576  0.053387  0.037447   
1093  0.029627  0.011868  0.022571  0.016818  0.017846  0.018985  0.028909   
1094  0.048898  0.022876   0.03951   0.02461  0.044201  0.045526  0.036347   

          7         8         9     ...      1085      1086      1087  \
0     0.223558  0.223558  0.275628  ...  0.024217  0.020864   0.03397   
1      0.10343   0.10343  0.182735  ...  0.010055  0.004837   0.02785   
2     0.223261  0.223261  0.301228  ...  0.047245  0.032326  0.047048   
3     0.109706  0.109706  0.205004  ...  0.006545  0.006867  0.026045   
4     0.187557  0.187557  0.265726  ...  0.014637   0.03588  0.039239   
...        ...       ...       ...  ...       ...       ...       ...   
1090  0.015769  0.015769  0.013655  ...  0.091536  0.090693  0.283405   
1091  0.023103  0.023103  0.009864  ...  0.042592  0.033986  0.123089   
1092  0.037518  0.037518  0.014907  ...  0.052095  0.047025  0.147413   
1093  0.011043  0.011043  0.039701  ...  0.055822  0.062793  0.218365   
1094  0.027485  0.027485  0.021838  ...  0.023248  0.019876  0.040885   

          1088      1089      1090      1091      1092      1093      1094  
0     0.016939   0.04833  0.021772  0.025764  0.034973  0.029627  0.048898  
1     0.002382  0.035186  0.000689  0.011381  0.027174  0.011868  0.022876  
2      0.01605  0.039024  0.017432  0.033008  0.044105  0.022571   0.03951  
3     0.008838  0.003713   0.00202  0.013693  0.020582  0.016818   0.02461  
4     0.048166  0.031589  0.040304  0.018786  0.021576  0.017846  0.044201  
...        ...       ...       ...       ...       ...       ...       ...  
1090  0.270455  0.104272       1.0  0.074116  0.048771  0.196541   0.01654  
1091  0.038774  0.072823  0.074116       1.0  0.054048  0.057026  0.024213  
1092  0.055245  0.119641  0.048771  0.054048       1.0  0.132541  0.021064  
1093  0.251933  0.176973  0.196541  0.057026  0.132541       1.0  0.019183  
1094  0.002636  0.008939   0.01654  0.024213  0.021064  0.019183       1.0  

[1095 rows x 1095 columns]

In [38]:
def sim_matrix(similarity_matrix, cluster1_id, cluster2_id):
    # 確保其收到的資料格式為int
    cluster1_id = int(cluster1_id)
    cluster2_id = int(cluster2_id)
    
    # 使用廣播找出兩個索引對應的最小相似度值
    similarity_matrix[cluster1_id, :] = np.minimum(similarity_matrix[cluster1_id, :], similarity_matrix[cluster2_id, :])
    similarity_matrix[:, cluster1_id] = similarity_matrix[cluster1_id, :]  # 利用對稱性更新列
    similarity_matrix[cluster1_id, cluster1_id] = -np.inf  # 設定 cluster1 內部的值為負無窮大
    
    # 將 cluster2 對應的行和列設為負無窮大，表示已經合併
    similarity_matrix[cluster2_id, :] = -np.inf
    similarity_matrix[:, cluster2_id] = -np.inf

In [39]:
def max_pos(test_C):
    max_position = np.unravel_index(np.argmax(test_C), test_C.shape) 
    return max_position

In [40]:
def apply_HAC(clusters, k):

    while k < len(test_C):
        k_max = max(max_pos(test_C))  
        k_min = min(max_pos(test_C))  
        clusters[k_min] += clusters[k_max] 
        clusters[k_max] = []  
        sim_matrix(test_C, k_min, k_max)  # 更新similarity matrix
        k += 1

    clusters = [group for group in clusters]  # 去除空的cluster
    
    return clusters

In [41]:
def write_file(cluster, k):
    file_name = f"./{k}.txt"
    with open(file_name, "w") as f:
        for group in cluster:
            if group:
                
                f.write('\n')
                f.write('\n'.join(map(str, sorted(group))))  
                f.write('\n')

In [42]:
# 分為8、13、20群
for k in [8, 13, 20]:
    output_k = k
    test_C = C.to_numpy()
    np.fill_diagonal(test_C, 0)  # 將sim matrix對角線值設為零

    clusters = [[i + 1] for i in range(len(test_C))]  # 將每個文章都分為一個cluster

    clusters = apply_HAC(clusters, k)

    for group in clusters:
        group.sort()  # 對每個群組的文章id進行排序

    # 寫入
    write_file(clusters, output_k)